In [1]:
!pip install rake_nltk
!pip install pymorphy3
!pip install pymorphy3-dicts-ru

# !pip uninstall numpy
# !pip install numpy==1.26.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 17.8 MB/s eta 0:00:00


In [ ]:
# !pip show numpy

In [ ]:
# !pip install surprise

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import ast
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import rake_nltk
from rake_nltk import Rake
import nltk

import re
from nltk.corpus import stopwords
import pymorphy3
# from surprise import Reader, Dataset, SVD, dump
# from surprise.model_selection import cross_validate
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
import pickle

# from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [4]:
movie_df = pd.read_csv('/content/drive/MyDrive/Kinopoisk/movie_df_small.csv')
# cast_df = pd.read_csv('/content/drive/MyDrive/Kinopoisk/cast_df.csv')

In [2]:
data = pd.read_csv('/content/drive/MyDrive/Kinopoisk/data.csv')

In [3]:
data= data.iloc[0:25000]

In [4]:
data

,Unnamed: 0,id,name,genres,persons,description,soup
0,0,298,Люди Икс 2,фантастика боевик триллер,брайансингер патрикстюарт хьюджекман иэнмаккел...,мутант продолжать борьба против общество котор...,брайансингер патрикстюарт хьюджекман иэнмаккел...
1,1,299,Матрица: Перезагрузка,фантастика боевик,лилливачовски киануривз лоренсфишбёрн хьюгоуив...,борец свобода нео тринити морфеус продолжать р...,лилливачовски киануривз лоренсфишбёрн хьюгоуив...
2,2,300,Лиззи Магуайр,мелодрама комедия приключения семейный музыка,джимфолл хиларидафф адамлэмберг халлиморган ро...,тринадцатилетний школьница лиззь магуайер прия...,джимфолл хиларидафф адамлэмберг халлиморган ро...
3,3,301,Матрица,фантастика боевик,лилливачовски киануривз лоренсфишбёрн кэрри-эн...,жизнь томас андерсон разделить два часть день ...,лилливачовски киануривз лоренсфишбёрн кэрри-эн...
4,4,302,Клад,драма комедия детектив приключения семейный,эндрюдэвис сигурниуивер джонвойт патришааркетт...,на семья подросток имя стенли илнетс прочесть ...,эндрюдэвис сигурниуивер джонвойт патришааркетт...
...,...,...,...,...,...,...,...
24995,44009,45605,"В Москве, проездом...",драма мелодрама комедия,ильягурин евгенийкарельских сергейшакуров нико...,четыре история короткий пребывание разный чело...,ильягурин евгенийкарельских сергейшакуров нико...
24996,44010,45606,В моей смерти прошу винить Клаву К.,драма мелодрама,эрнестясан владимиршевельков надеждагоршкова е...,сереж четыре год он полюбить клава первый взгл...,эрнестясан владимиршевельков надеждагоршкова е...
24997,44011,45607,В мертвой петле,драма биография,суламифьцыбульник олегстриженов николайпишвано...,биографический фильм знаменитый русский лётчик...,суламифьцыбульник олегстриженов николайпишвано...
24998,44012,45608,В начале игры,драма спорт,юриймастюгин андрейсмоляков пётрвельяминов рим...,виктор круглов молодой футболист провинциальны...,юриймастюгин андрейсмоляков пётрвельяминов рим...


In [5]:
print(movie_df.shape[0])
movie_df=movie_df.dropna(subset=['name', 'persons','genres', 'description'])
print(movie_df.shape[0])

15798
15798


In [6]:
movie_df[['name', 'persons', 'description']]

,name,persons,description
0,Люди Икс 2,"[{'id': 8214, 'photo': 'https://st.kp.yandex.n...","Мутанты продолжают борьбу против общества, кот..."
1,Матрица: Перезагрузка,"[{'id': 7836, 'photo': 'https://st.kp.yandex.n...","Борцы за свободу Нео, Тринити и Морфеус продол..."
2,Лиззи Магуайр,"[{'id': 31505, 'photo': 'https://image.openmov...",Тринадцатилетняя школьница Лиззи Магуайер и ее...
3,Матрица,"[{'id': 7836, 'photo': 'https://image.openmovi...",Жизнь Томаса Андерсона разделена на две части:...
4,Клад,"[{'id': 6915, 'photo': 'https://image.openmovi...",На семье подростка по имени Стенли Илнетс (про...
...,...,...,...
15793,Все дело в воде,"[{'id': 33276, 'photo': 'https://image.openmov...",Действие фильма происходит в маленьком консерв...
15794,Слизни,"[{'id': 172536, 'photo': 'https://image.openmo...",В небольшом городке начали исчезать жители и п...
15795,Замужество Марии Браун,"[{'id': 93249, 'photo': 'https://image.openmov...","Проводив мужа на Восточный фронт, Мария начина..."
15796,В год тринадцати лун,"[{'id': 87320, 'photo': 'https://st.kp.yandex....",«Каждый год - это год Луны» - говорится в тума...


In [7]:
def main_cast(persons):
  persons = ast.literal_eval(persons)
  main_cast = []
  director_name = ''
  for item in persons:
    if item['name'] != None:
      main_cast.append(item['name'].replace(' ', ''))
    if item['enProfession'] == 'director':
      director_name = item['name']
  if len(main_cast) >=6:
    main_cast = main_cast[:6:]
  else:
    main_cast = main_cast[:len(main_cast):]
  if director_name != None:
    main_cast.insert(0, director_name.replace(' ', ''))
  else:
    main_cast.insert(0, 'unknown')
  return ' '.join(main_cast)

In [8]:
def genres_cleanup(genres):
  genres = ast.literal_eval(genres)
  genres_list = []
  for item in genres:
    genres_list.append(item['name'])
    # print(item)
  return ' '.join(genres_list)

In [ ]:

# def description_cleanup(tokens):
#   morph = pymorphy3.MorphAnalyzer()
#   patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
#   key_words = []
#   tokens = nltk.tokenize.word_tokenize(tokens)
#   print(tokens)
#   for token in tokens:
#     token = re.sub(patterns, '', token)
#     if token not in stopwords.words("russian"):
#       if morph.parse(token)[0].normal_form != '':
#         key_words.append(morph.parse(token)[0].normal_form)
#         print(morph.parse(token)[0].normal_form)
#   print(stopwords.words("russian"))
#   return key_words

#   print(description_cleanup(movie_df.at[358, 'description']))

In [9]:
def description_cleanup(orig_tokens):
  try:
    morph = pymorphy3.MorphAnalyzer()
    patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
    key_words = []
    tokens = nltk.tokenize.word_tokenize(orig_tokens)
    # print(tokens)
    for token in tokens:
      token = re.sub(patterns, '', token)
      if token not in stopwords.words("russian"):
        if morph.parse(token)[0].normal_form != '':
          key_words.append(morph.parse(token)[0].normal_form)
          # print(morph.parse(token)[0].normal_form)
      # print(stopwords.words("russian"))
  except:
    print(orig_tokens)
    return 'unknown'
  return ' '.join(key_words)

In [10]:
data = movie_df[['id','name', 'genres', 'persons', 'description']]
data.loc[:,'persons'] = data['persons'].apply(main_cast)
data.loc[:, 'persons'] = data['persons'].str.lower()
data.loc[:, 'genres'] = data['genres'].apply(genres_cleanup)
data.loc[:, 'description'] = data['description'].apply(description_cleanup)
# data.drop(columns='persons', inplace=True)

In [11]:
data['soup'] = data['persons']+' '+ data['genres'] + ' ' + data['description']

<ipython-input-11-19b494aae7d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['soup'] = data['persons']+' '+ data['genres'] + ' ' + data['description']


In [12]:
data.to_csv('/content/drive/MyDrive/Kinopoisk/data.csv')

In [5]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2), min_df=0.0)


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfid = TfidfVectorizer(
    min_df = 3,
    max_features = None,
    analyzer = "word",
    ngram_range = (1,3)
)

tfid_matrix = tfid.fit_transform(data['soup'])

tfid_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 919031 stored elements and shape (15798, 58234)>

In [5]:
from sklearn.metrics.pairwise import sigmoid_kernel

sig = sigmoid_kernel(tfid_matrix, tfid_matrix)

sig[0]

array([0.76159887, 0.76159433, 0.76159417, ..., 0.76159416, 0.76159416,
       0.76159416])

In [6]:
count_matrix = count.fit_transform(data['soup'])

In [14]:
count_matrix

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 1000875 stored elements and shape (10000, 453048)>

In [7]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [14]:
cosine_sim = cosine_similarity(tfid_matrix, tfid_matrix)

In [6]:
cosine_sim = sig

In [15]:

with open('/content/drive/MyDrive/Kinopoisk/cosine_sim.npy', 'wb') as f:
  np.save(f, cosine_sim)

In [16]:
data = data.reset_index()
names = data['name']
indices = pd.Series(data.index, index=data['name'])

In [17]:
def get_recommendations(idx):
    idx = indices[idx]
    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    print(sim_scores)
    movie_indices = [i[0] for i in sim_scores]
    return names.iloc[movie_indices]
    # return movie_indices

In [19]:
get_recommendations(1)

[(17, np.float64(0.37248735797445554)), (3, np.float64(0.3526405509954558)), (2356, np.float64(0.09776597768034022)), (13812, np.float64(0.08206524583050828)), (2355, np.float64(0.08084147607525144))]


<ipython-input-17-82bed7fdca1e>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  idx = indices[idx]


,name
17,Матрица: Революция
3,Матрица
2356,Красная планета
13812,Катастрофа роботов
2355,Охотник на убийц


In [20]:
# data = data.reset_index()
ids = data['id']
with open('/content/drive/MyDrive/Kinopoisk/ids.pkl', 'wb') as f:
  pickle.dump(ids, f)


indices = pd.Series(data.index, index=data['id'])
with open('/content/drive/MyDrive/Kinopoisk/indices.pkl', 'wb') as f:
  pickle.dump(indices, f)


In [ ]:
idx = 298
idx = indices[idx]
print(idx)
sim_scores = list(enumerate(cosine_sim[idx]))
print(sim_scores)
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[1:10]
movie_indices = [i[0] for i in sim_scores]
ids.iloc[movie_indices]

0
[(0, np.float64(1.0000000000000009)), (1, np.float64(0.06048207481268218)), (2, np.float64(0.006637086926267733)), (3, np.float64(0.027403260268473117)), (4, np.float64(0.01607545306456218)), (5, np.float64(0.010782650440285159)), (6, np.float64(0.039835147194890626)), (7, np.float64(0.021744269101781406)), (8, np.float64(0.10437827457863963)), (9, np.float64(0.0)), (10, np.float64(0.0)), (11, np.float64(0.05556761580872173)), (12, np.float64(0.0199112607788032)), (13, np.float64(0.03012974308618838)), (14, np.float64(0.04678621783702883)), (15, np.float64(0.011034241346230916)), (16, np.float64(0.0)), (17, np.float64(0.03945661007840242)), (18, np.float64(0.0)), (19, np.float64(0.04348853820356282)), (20, np.float64(0.05145209776622857)), (21, np.float64(0.007922298864966105)), (22, np.float64(0.0742994887670994)), (23, np.float64(0.0)), (24, np.float64(0.03256555965659629)), (25, np.float64(0.03827963277575087)), (26, np.float64(0.0)), (27, np.float64(0.02539183580595128)), (28, np

,id
586,886
9883,13995
11370,16718
434,734
11001,15946
11204,16417
15406,24196
4857,6447
16182,26366


In [ ]:
with open('/content/drive/MyDrive/Kinopoisk/cosine_sim.npy', 'rb') as f:
  cosine_sim = np.load(f)


In [ ]:
def get_recommendations(idx):
    idx = indices[idx]
    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    print(sim_scores)
    movie_indices = [i[0] for i in sim_scores]
    return ids.iloc[movie_indices]

In [ ]:
indices = pd.Series(movie_df.index, index=data['id'])
ids = data['id']
get_recommendations(1943)

[(11264, np.float64(0.1453315231796812)), (11424, np.float64(0.1432229748078866)), (85, np.float64(0.13903017140231752)), (13608, np.float64(0.13879558367189768)), (14278, np.float64(0.1344365610500929))]


,id
11264,11764
11424,11927
85,384
13608,14139
14278,14814


In [ ]:
type(indices)

pandas.core.series.Series

In [ ]:
data.iloc[rec_movies.iloc[4]]

/usr/local/lib/python3.11/dist-packages/pandas/core/series.py:1031: RuntimeWarning: invalid value encountered in cast
  arr = np.asarray(values, dtype=dtype)


IndexError: positional indexers are out-of-bounds

In [ ]:
review_df = pd.read_csv('/content/drive/MyDrive/Kinopoisk/review_df.csv')

In [ ]:
# user = review_df['authorId'].sample(1, random_state = 3).iloc[0]
user = 1597999

In [ ]:
rec_movies = review_df.loc[(review_df['authorId'] == user) & (review_df['type'] =='Позитивный')].sort_values('date', ascending = False).iloc[:5]['movieId'].apply(get_recommendations)

[(1289, np.float64(0.11083777463036817)), (777, np.float64(0.10057523818729522)), (1533, np.float64(0.09912073072209225)), (6225, np.float64(0.0990990990990991)), (1444, np.float64(0.09610975593794979))]
[(7138, np.float64(0.1258817936510608)), (11769, np.float64(0.11986403764995548)), (4329, np.float64(0.11758894715842626)), (12554, np.float64(0.11602056973036094)), (2820, np.float64(0.11236012954671795))]
[(10090, np.float64(0.16848017885865282)), (5032, np.float64(0.1434634080478473)), (241, np.float64(0.14244928722116237)), (13691, np.float64(0.13799996964001005)), (113, np.float64(0.134637813499663))]
[(2525, np.float64(0.18276863209719615)), (3743, np.float64(0.17487340036763677)), (2748, np.float64(0.15911219243888639)), (4003, np.float64(0.15527001643447583)), (817, np.float64(0.15051153338813572))]
[(3503, np.float64(0.14674168715023575)), (13045, np.float64(0.14491645112472712)), (9379, np.float64(0.1322330306735767)), (11282, np.float64(0.12926112238824689)), (9946, np.float

In [ ]:
# ratings = review_df[['authorId', 'movieId', 'type']]

In [ ]:
# ratings.loc[:,'type'] = ratings['type'].map({'Позитивный':3, 'Нейтральный':2, 'Негативный':1})
# ratings = ratings[['authorId', 'movieId', 'type']]
# ratings.rename(columns={'type':'rating', 'name':'title', 'author':'user'}, inplace=True)
# ratings

In [ ]:
# ratings = ratings[ratings.groupby('movieId').transform('size') > 10]

In [ ]:
# from csv import reader
# reader = Reader()

In [ ]:
# data = Dataset.load_from_df(ratings[['authorId', 'movieId', 'rating']], reader)

In [ ]:
# svd = SVD()

# cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
# trainset = data.build_full_trainset()
# svd.fit(trainset)

In [ ]:
# dump.dump('/content/drive/MyDrive/Kinopoisk/svd_dump_file', algo=svd)

In [ ]:
# def hybrid(userId, title):
#     idx = indices[title]
#     tmdbId = id_map.loc[title]['id']
#     #print(idx)
#     movie_id = id_map.loc[title]['id']

#     sim_scores = list(enumerate(cosine_sim[int(idx)]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     sim_scores = sim_scores[1:15]
#     movie_indices = [i[0] for i in sim_scores]

#     movies = movie_df.iloc[movie_indices][['name', 'rating', 'id']]

#     movies['est'] = movies['id'].apply(lambda x: svd.predict(userId,x).est)
#     movies = movies.sort_values('est', ascending=False)
#     return movies.head(10)

In [23]:
data.to_csv('/content/drive/MyDrive/Kinopoisk/data.csv')